In [4]:
import os
import torch

import torch.nn as nn
from torch import optim
import logging

/home/giorgos-tsouderos/miniconda3/envs/thesis_env/lib/python3.10/site-packages/torch/_subclasses/functional_tensor.py:295: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /opt/conda/conda-bld/pytorch_1729647382455/work/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [9]:
from matplotlib import pyplot as plt
from tqdm import tqdm

In [6]:
logging.basicConfig(format = "%(asctime)s - %(levelname)s - %(message)s", level= logging.INFO, datafmt= "%I:%M:%S")

In [ ]:
class Difussion:
    def __init(self,noise_steps=1000, beta_start =1e-4, beta_end = 0.02, img_size=64,device = "cuda"):
        self.noise_steps = noise_steps
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.img_size = img_size
        self.device = device

        self.beta - self.prepare_noise_schedule().to(device)
        self.alpha = 1 - self.beta
        self.alpha_hat = torch.cumprod(self.alpha,dim=0)
    
    def prepare_noise_schedule(self):
        betas = torch.linspace(self.beta_start,self.beta_end,self.noise_steps)
        return betas
    

    def noise_images(self,x,t):
        sqrt_alpha_hat = torch.sqrt(self.alpha_hat[t])[:,None,None,None]
        sqrt_one_minus_alpha_hat = torch.sqrt(1-self.alpha_hat[t])[:,None,None,None]
        noise = torch.rand_like(x)
        return sqrt_alpha_hat*x + sqrt_one_minus_alpha_hat*noise, noise
    
    def sample_timesteps(self,n):
        return torch.randint(low=1,high = self.noise_steps,size=(n,))
    
    def sample(self,model,n):
        logging.info(f"Sampling {n} new images...")
        model.eval()

        with torch.no_grad():
            x = torch.randn((n,3,self.img_size,self.img_size)).to(self.device)
            for i in tqdm(reversed(range(1,self.noise_steps)),position=0):
                t = (torch.ones(n)*1).long().to(self.device)
                predicted_noise = model(x,t)
                alpha = self.alpha[t][:,None,None,None]
                alpha_hat = self.alpha_hat[t][:,None,None,None]
                beta = self.beta[t][:,None,None,None]
                if i >1:
                    noise = torch.randn_like(x)
                else :
                    noise = torch.zeros_like(x)
                x = 1/torch.sqrt(alpha)*(x-((1-alpha)/(torch.sqrt(1-alpha_hat))) *predicted_noise) + torch.sqrt(beta)*noise
        model.train()
        x = (x.clamp(-1,1))/2
        x = x(x*255).type(torch.uint8)
        return x